## 丢弃法

除了前一节介绍的权重衰减以外，深度学习模型常常使用丢弃法$（dropout）$ 来应对过拟合问题。丢弃法有一些不同的变体。本节中提到的丢弃法特指倒置丢弃法$（inverted dropout）$。

在3.8节中描述的多层感知机的计算表达式如下：
<center>
    $ h_i= \phi(x_1w_{1i}+x_2w_{2i}+x_3w_{3i}+x_4w_{4i}+b_i)$
</center>
当对该隐藏层使用丢弃法的时，该层的隐藏单元将有一定概率被丢弃掉。设丢弃概率为$p$，那么有$p$的概率$h_i$会被清零，有$1-p$的概率$h_i$会除以$1-p$做拉伸。丢弃概率是丢弃法的超参数。具体来说，设随机变量${\xi}_i$为$0$和$1$的概率分别为$p$和$1-p$。使用丢弃法时我们计算新的隐藏单元$h_{inew}$
<center>
    $ h_{inew} =  \frac{\xi}{1-p}h_i$
</center>
由于$E({\xi}_i)=1-p$，因此
<center>
    $ E(h_{inew})=\frac{E({\xi}_i)}{1-p}h_i=h_i $
</center>
即**丢弃法不改变其输入的期望值**。

由于在训练中隐藏层神经元的丢弃是随机的，即$h_1,…,h_5$ 都有可能被清零，输出层的计算无法过度依赖$h_1,…,h_5$中的任一个，从而在训练模型时起到正则化的作用，并可以用来应对过拟合。

在**测试**模型时，我们为了拿到更加确定性的结果，一般不使用丢弃法。

**从零开始实现**

In [1]:
%matplotlib inline
import torch
import numpy as np
import torch.nn as nn
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l

def dropout(X, drop_prob):
    X=X.float()
    assert 0 <= drop_prob <=1
    keep_prob = 1-drop_prob
    
    if keep_prob == 0:
        return torch.zeros_like(X)
    mask = (torch.rand(X.shape)<keep_prob).float()
    
    return mask*X/keep_prob

In [2]:
X = torch.arange(16).view(2,8)
# print(torch.rand(X.shape)<0.5)
dropout(X, 0)

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])

In [3]:
dropout(X, 0.5)

tensor([[ 0.,  0.,  0.,  0.,  8.,  0., 12., 14.],
        [16.,  0.,  0.,  0., 24.,  0., 28.,  0.]])

In [4]:
dropout(X, 1.0)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [5]:
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10,256,256

w1 = torch.tensor(np.random.normal(0,0.01,size=(num_inputs,num_hiddens1)),dtype=torch.float,requires_grad=True)
b1 = torch.zeros(num_hiddens1,dtype=torch.float,requires_grad=True)

w2 = torch.tensor(np.random.normal(0,0.01,size=(num_hiddens1,num_hiddens2)),dtype=torch.float,requires_grad=True)
b2 = torch.zeros(num_hiddens2,dtype=torch.float,requires_grad=True)

w3 = torch.tensor(np.random.normal(0,0.01,size=(num_hiddens2,num_outputs)),dtype=torch.float,requires_grad=True)
b3 = torch.zeros(num_outputs,dtype=torch.float,requires_grad=True)

params = [w1,b1,w2,b2,w3,b3]

**定义模型**

下面定义的模型将全连接层和激活函数$ReLU$串起来，并对每个激活函数的输出使用丢弃法。我们可以分别设置各个层的丢弃概率。通常的建议是把$靠近输入层$的丢弃概率设得小一点。

In [6]:
drop_prob1, drop_prob2 = 0.2, 0.5
def net(X, is_training=True):
    X = X.view(-1,num_inputs)
    H1 = (torch.mm(X,w1)+b1).relu()
    if is_training:
        H1 = dropout(H1, drop_prob1)
    H2 = (torch.mm(H1,w2)+b2).relu()
    if is_training:
        H2 = dropout(H2, drop_prob2)
    return torch.mm(H2,w3)+b3

In [7]:
epochs, lr, batch_size = 5, 100.0, 256
loss = torch.nn.CrossEntropyLoss()
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size,root="D:\\FYC\\ict\\Datasets\\FashionMNIST")
d2l.train_ch3(net, train_iter, test_iter, loss, epochs, batch_size, params, lr)

epoch 1, loss 0.0046, train acc 0.542, test acc 0.762
epoch 2, loss 0.0023, train acc 0.783, test acc 0.798
epoch 3, loss 0.0019, train acc 0.820, test acc 0.805
epoch 4, loss 0.0017, train acc 0.838, test acc 0.816
epoch 5, loss 0.0016, train acc 0.847, test acc 0.819


**简洁实现**

In [8]:
net = nn.Sequential(
    d2l.FlattenLayer(),
    nn.Linear(num_inputs,num_hiddens1),
    nn.ReLU(),
    nn.Dropout(drop_prob1),
    nn.Linear(num_hiddens1,num_hiddens2),
    nn.ReLU(),
    nn.Dropout(drop_prob2),
    nn.Linear(num_hiddens2,num_outputs),
)
for param in params:
    nn.init.normal_(param, mean=0, std=0.01)

In [10]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)
d2l.train_ch3(net, train_iter, test_iter, loss, epochs, batch_size, None, None, optimizer)

epoch 1, loss 0.0035, train acc 0.668, test acc 0.778
epoch 2, loss 0.0021, train acc 0.805, test acc 0.830
epoch 3, loss 0.0018, train acc 0.833, test acc 0.818
epoch 4, loss 0.0017, train acc 0.844, test acc 0.819
epoch 5, loss 0.0016, train acc 0.855, test acc 0.845


我们可以通过使用丢弃法应对过拟合。

**丢弃法只在训练模型时使用。**